In [ ]:
from bs4 import BeautifulSoup
import lxml
import requests
import pandas as pd

data = []
a = 1
site = 'vatan'

while a <= 20:
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}
    r = requests.get("https://www.vatanbilgisayar.com/notebook/?gclid=Cj0KCQjwkt6aBhDKARIsAAyeLJ3EURkWFx9yZ5TJOT5eTK2TstDcuvX2NYtC3A21e2xpP9Et4qWBOeMaAvRnEALw_wcB", headers=headers)
    soup = BeautifulSoup(r.content, "lxml")

    div_wrapper = soup.find("div", attrs={"class": "wrapper-product-main"})
    div_products = div_wrapper.find("div", attrs={"id": "productsLoad"})
    product_list = div_products.find_all("div", attrs={"class": "product-list product-list--list-page"})

    for details in product_list:
        link_end = details.a.get("href")
        link_base = "https://www.vatanbilgisayar.com"
        link = link_base + link_end
        print(link)
        r1 = requests.get(link, headers=headers)
        soup1 = BeautifulSoup(r1.content, "lxml")
        soup2 = BeautifulSoup(r1.text, 'html.parser')

        product_price = soup1.find("span", attrs={"class": "product-list__price"}).text
        product_price = product_price + "TL"
        print(product_price)

        product_brand = soup1.find("h1", attrs={"class": "product-list__product-name"}).text.split()
        print(product_brand[0])

        table = soup2.find('div', class_='row masonry-tab')

        for detail in table.find_all('tr'):
            row_label = detail.find('td').text

            if row_label == 'Ekran Boyutu':
                screen_size = detail.find('p').text.strip().replace("inch", "")
                print(screen_size)

            if row_label == 'Disk Kapasitesi':
                disk_size = detail.find('p').text
                print(disk_size)

            if row_label == 'Ram (Sistem Belleği)':
                ram_size = detail.find('p').text.strip().replace(" GB", "").split()
                print(ram_size[0])

            if row_label == 'İşlemci Nesli':
                processor_generation = detail.find('p').text.strip().replace("AMD ", "").strip().replace("Intel ", "")
                print(processor_generation)

            if row_label == 'İşlemci Teknolojisi':
                processor_type = detail.find('p').text.strip().replace("Core ", "").strip().replace("™", "")
                print(processor_type)

            if row_label == 'İşletim Sistemi':
                os = detail.find('p').text.strip().split()
                if os[0] == 'FreeDOS':
                    os[0] = 'None'
                    print(os[0])
                else:
                    print(os[0])

        data.append([product_price, product_brand[0], screen_size, disk_size, ram_size[0], processor_generation, processor_type, os[0], site])
    a = a + 1

In [13]:
df = pd.DataFrame(data)
df.columns = ["product_price", "brand", "screen_size", "disk_size", "ram", "processor_generation", "processor_type", "os", "site"]
df.to_excel("vatan1.xlsx")
